### 라이브러리 설치

In [1]:
!pip install ortools
!pip install xlwt
!pip install networkx

     |████████████████████████████████| 14.4 MB 864 kB/s eta 0:00:01
  Using cached protobuf-3.15.8-cp38-cp38-manylinux1_x86_64.whl (1.0 MB)
  Using cached absl_py-0.12.0-py3-none-any.whl (129 kB)


### 데이터 불러오기

왼쪽 파일 업로드 기능을 이용해서 엑셀 파일을 업로드해주세요.

파일명은 navi_data.xlsx 이고, 액티비트 목록은 '마감작업'시트에, 작업생산성은 '작업생산성'시트에, 선후관계는 '선후관계' 시트에서 불러옵니다

In [3]:
import pandas as pd

jobs_df = pd.read_excel('navi_data.xlsx','그리드별작업리스트')
jobs_df = jobs_df[jobs_df['grid'] != 'grid']  # index row is repeated
print(len(jobs_df))
jobs_df.head()
#jobs_df.iloc[0]

36


,grid,작업
0,A1,철거
1,A1,마킹
2,A1,칸막이A
3,A1,칸막이B
4,A1,방통


In [4]:
productivity_df = pd.read_excel('navi_data.xlsx','업무템플릿')
print(len(productivity_df))
productivity_df.head()

10


,Code,작업,생산성,선행완료
0,100,철거,1,1
1,110,마킹,2,2
2,120,칸막이A,1,1
3,130,칸막이B,2,3
4,140,방통,3,4


In [5]:
productivity_per_code = {}

for row in productivity_df.itertuples():
    productivity_per_code[row.Code] = row.생산성
len(productivity_per_code)

productivity_per_code

{100: 1,
 110: 2,
 120: 1,
 130: 2,
 140: 3,
 150: 1,
 160: 2,
 170: 1,
 180: 2,
 190: 1}

In [35]:
import collections
codes_per_area_type = collections.defaultdict(list)

for row in jobs_df.itertuples():
    codes_per_area_type[row.grid].append(row.작업)
    
# 중복검사
for area_type, codes in codes_per_area_type.items():
    if len(set(codes)) != len(codes):
        print(f'그리드 {area_type}에서 오류:')
        for code, count in collections.Counter(codes).items():
            if count < 2:
                continue
            print(f'- 코드 {code}가 {count}번 중복되었습니다.')


In [38]:
# Parse jobs 

jobs = []
code_to_job_id = {}
for row in jobs_df.itertuples():
    code = row.Code
    name = row.작업
    if code in code_to_job_id:
        # 이미 등록된 코드이면 작업명이 같아야함
        assert jobs[code_to_job_id[code]]['작업'] == name, code
    else:
        code_to_job_id[code] = len(jobs)
        jobs.append({
            '작업': name,
            'Code': code,
            '생산성': productivity_per_code[code],  # temp
            'requires': [],
        })
len(jobs)


"\njobs = []\ncode_to_job_id = {}\nfor row in jobs_df.itertuples():\n    code = row.Code\n    name = row.작업\n    if code in code_to_job_id:\n        # 이미 등록된 코드이면 작업명이 같아야함\n        assert jobs[code_to_job_id[code]]['작업'] == name, code\n    else:\n        code_to_job_id[code] = len(jobs)\n        jobs.append({\n            '작업': name,\n            'Code': code,\n            '생산성': productivity_per_code[code],  # temp\n            'requires': [],\n        })\nlen(jobs)\n"

In [39]:
constraints_df = pd.read_excel('navi_data.xlsx','선후관계템플릿')
print(len(constraints_df))

# FS, SS, FF 관계 모두 FS로 처리함.
constraints_df.head()

9


,선행,후행
0,철거,마킹
1,마킹,칸막이A
2,칸막이A,칸막이B
3,칸막이B,방통
4,방통,천정


In [43]:
# Parse constraints

# for row in constraints_df.itertuples():
#     if row.후행 not in code_to_job_id:
#        print(f'{row.후행} 코드가 존재하지 않지만 무시합니다')
#        continue
#     if row.선행 not in code_to_job_id:
#        print(f'{row.선행} 코드가 존재하지 않지만 무시합니다')
#        continue
#     jobs[code_to_job_id[row.후행]]['requires'].append(code_to_job_id[row.선행])
    
# for job in jobs:
#     # require condition 중복검사
#     # SS도 있고 FF도 있어서 중복가능
#     # assert len(set(job['requires'])) == len(job['requires']), job
#     job['requires'] = list(set(job['requires']))
    
# # number of constraints
# sum(len(job['requires']) for job in jobs)

마킹 코드가 존재하지 않지만 무시합니다
칸막이A 코드가 존재하지 않지만 무시합니다
칸막이B 코드가 존재하지 않지만 무시합니다
방통 코드가 존재하지 않지만 무시합니다
천정 코드가 존재하지 않지만 무시합니다
바닥 코드가 존재하지 않지만 무시합니다
도어 코드가 존재하지 않지만 무시합니다
가구 코드가 존재하지 않지만 무시합니다
가전 코드가 존재하지 않지만 무시합니다


0

### 선후관계 사이클 조사

선후관계가 순환되어 모순되어있는 경우를 조사합니다. 있다면, 선후관계 중 순환되는 부분을 끊어주어야 합니다.

In [45]:

#jobs[0]

In [46]:
# detect cycle
# import networkx as nx

# edges = [(j0, j1) for j0, job in enumerate(jobs) for j1 in job['requires']]

# try:
#     while 1:
#         # cycle 이 없을 때까지 cycle 의 첫 부분을 끊으면서 출력
#         G = nx.DiGraph(edges)
#         cycle = nx.find_cycle(G, orientation='original')
#         print('순환 발견: ')
#         for edge in cycle:
#             job0, job1 = jobs[edge[0]], jobs[edge[1]]
#             print(f"-  {job0['code']}({job0['name']}) < {job1['code']}({job1['name']})")
#         edge_to_remove = (cycle[0][0], cycle[0][1])
#         edges.remove(edge_to_remove)

# except nx.exception.NetworkXNoCycle:
#     pass

In [21]:
# jobs[78]['requires'].remove(79)
# jobs[32]['requires'].remove(33)
# jobs[21]['requires'].remove(34)

In [25]:
#area_types = {
#    '오피스': ['A1', 'A2','A3','A4','B1','B2','B3','B4','C1','C2'],
#    '코어': ['C3','C4'],
#    '휴게실': ['D1','D2'],
#    '화장실': ['D3','D4'],
#}


In [47]:
areas = []
# convert job code to job id
job_id_per_area_type = {area_type: [code_to_job_id[code] for code in codes] for area_type, codes in codes_per_area_type.items()}
for area_type, area_names in area_types.items():
    for area_name in area_names:
        areas.append({
            'name': area_name,
            'type': area_type,
            'jobs': job_id_per_area_type[area_type]
        })
        
jobs_per_area = [
    area['jobs']
    for area in areas
]
len(jobs_per_area)

KeyError: '철거'

In [28]:

for ids in jobs_per_area:
    print(' '.join(f'{i:2}' for i in ids))

 0 41 42 43 44 45 46 47 48 49 50  7  8  9 10 11 12 14 15 16 17 18 20 21 22 23 24 25 26 51 52 53 54 33 55 40 35
 0 41 42 43 44 45 46 47 48 49 50  7  8  9 10 11 12 14 15 16 17 18 20 21 22 23 24 25 26 51 52 53 54 33 55 40 35
 0 41 42 43 44 45 46 47 48 49 50  7  8  9 10 11 12 14 15 16 17 18 20 21 22 23 24 25 26 51 52 53 54 33 55 40 35
 0 41 42 43 44 45 46 47 48 49 50  7  8  9 10 11 12 14 15 16 17 18 20 21 22 23 24 25 26 51 52 53 54 33 55 40 35
 0 41 42 43 44 45 46 47 48 49 50  7  8  9 10 11 12 14 15 16 17 18 20 21 22 23 24 25 26 51 52 53 54 33 55 40 35
 0 41 42 43 44 45 46 47 48 49 50  7  8  9 10 11 12 14 15 16 17 18 20 21 22 23 24 25 26 51 52 53 54 33 55 40 35
 0 41 42 43 44 45 46 47 48 49 50  7  8  9 10 11 12 14 15 16 17 18 20 21 22 23 24 25 26 51 52 53 54 33 55 40 35
 0 41 42 43 44 45 46 47 48 49 50  7  8  9 10 11 12 14 15 16 17 18 20 21 22 23 24 25 26 51 52 53 54 33 55 40 35
 0 41 42 43 44 45 46 47 48 49 50  7  8  9 10 11 12 14 15 16 17 18 20 21 22 23 24 25 26 51 52 53 54 33 55 40 35
 

In [21]:
from ortools.sat.python import cp_model

CP_STATUS = {
    cp_model.UNKNOWN: 'unknown',
    cp_model.MODEL_INVALID: 'model_invalid',
    cp_model.FEASIBLE: 'feasible',
    cp_model.INFEASIBLE: 'infeasible',
    cp_model.OPTIMAL: 'optimal',
}

In [22]:
import datetime

# gantt chart 라이브러리 dhtmlx에서 쓰이는 날짜 방식
def format_date(d: datetime.datetime):
    return d.strftime('%d-%m-%Y')
format_date(datetime.datetime.now())

'09-04-2021'

In [23]:
import json

# json dump할 때 date 객체를 js Date객체로 바꿔주는 인코더
class DateJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, datetime.date):
            return obj.strftime('@@new Date(%Y, %m, %d)@@')
        else:
            return super(DateJSONEncoder, self).default(obj)

json.dumps({'d': datetime.date.today()}, cls=DateJSONEncoder)

'{"d": "@@new Date(2021, 04, 09)@@"}'

In [24]:
import math

model = cp_model.CpModel()


jobs_count = len(jobs)
area_count = len(jobs_per_area)

start_date_constraints = []

horizon = 120
objective_makespan = 0  # stops after hitting this goal

# Create variables
T = {
    (j, s): model.NewIntVar(0, horizon, f'T({j}, {s})')
        for s in range(area_count) for j in jobs_per_area[s] 
}

Start = {
    j: model.NewIntVar(0, horizon, f'Start({j})') for j in range(jobs_count)
}

# set user-defined additinoal constraints
if start_date_constraints:
    for j, start_date_offset in start_date_constraints.items():
        model.Add(Start[j] >= start_date_offset)


NameError: name 'jobs' is not defined

In [33]:
        
# set time limits per jobs
total_workarea = {
    j: sum(j == j_ for js in jobs_per_area for j_ in js) for j in range(jobs_count)
}
timelimit = {
    j: math.ceil(total_workarea[j] / job['productivity']) for j, job in enumerate(jobs)
}

# Create derived variables for calculation.
x = {
    # !! DIFFERENT definition than above
    # x[t, j, s] == True iff t == T[j, s] + start[j]
    (t, j, s): model.NewBoolVar(f'x({t}, {j}, {s}')
        for s in range(area_count) for j in jobs_per_area[s] for t in range(timelimit[j])
}
for s in range(area_count):
    for j in jobs_per_area[s]:
        for t in range(timelimit[j]):
            model.Add(T[j,s] == t + Start[j]).OnlyEnforceIf(x[t, j, s])
            model.Add(T[j,s] != t + Start[j]).OnlyEnforceIf(x[t, j, s].Not())


In [34]:

# define workload variable. productivity is set here.
Workload = {
    (j, t): model.NewIntVar(1, job['productivity'], f'Workload({j}, {t})')
        for j, job in enumerate(jobs) for t in range(timelimit[j])
}

# define relationship between Workload and x
for j in range(jobs_count):
    for t in range(timelimit[j]):
        model.Add(sum(x[t, j, s] for s in range(area_count) if j in jobs_per_area[s]) == Workload[j, t])




In [35]:

# Total workarea constraint for workload
for j in range(jobs_count):
    model.Add(sum(Workload[j, t] for t in range(timelimit[j])) == total_workarea[j])


# Create Precedency Constraints
for s in range(area_count):
    for j in jobs_per_area[s]:
        job = jobs[j]
        for require in job['requires']:
            if require not in jobs_per_area[s]:
                # skip if precendecy is not worked on this area
                continue
            model.Add(T[require, s] < T[j, s])


In [36]:

# Create immediate precedency constraint
for s in range(area_count):
    for j in jobs_per_area[s]:
        job = jobs[j]
        preceeding_job = job.get('follows')
        if not preceeding_job or preceeding_job not in jobs_per_area[s]:
            continue
        model.Add(T[preceeding_job, s] + 1 == T[j, s])

# No two jobs done at the same time at the same place
for s in range(area_count):
    model.AddAllDifferent([T[j, s] for j in jobs_per_area[s]])


In [37]:

# Makespan objective.
makespan = model.NewIntVar(0, horizon, 'makespan')
model.AddMaxEquality(makespan, [T[j, s] for s in range(area_count) for j in jobs_per_area[s]])
model.Minimize(makespan)

# Register solution callback for early stopping
class SolutionCallback(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, objective):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__objective = objective

    def on_solution_callback(self):
        current_makespan = self.Value(self.__objective) + 1
        print(f'Current makespan: {current_makespan}')
        if objective_makespan and current_makespan <= objective_makespan:
            self.StopSearch()


# Solve model.
#started_at = datetime.datetime.now()
#solver = cp_model.CpSolver()
status = solver.SolveWithSolutionCallback(model, SolutionCallback(makespan))
ended_at = datetime.datetime.now()
took_seconds = (ended_at - started_at).total_seconds()
print(f'solver status: {CP_STATUS[status]}')
print(f'Took {took_seconds} seconds')

optimal_makespan = int(solver.ObjectiveValue()) + 1
if status == cp_model.OPTIMAL:
    print(f'Optimal makespan: {optimal_makespan}')

solver status: infeasible
Took 0.790071 seconds


In [51]:
# Solve model.
started_at = datetime.datetime.now()
solver = cp_model.CpSolver()
status = solver.SolveWithSolutionCallback(model, SolutionCallback(makespan))
ended_at = datetime.datetime.now()
took_seconds = (ended_at - started_at).total_seconds()
print(f'solver status: {CP_STATUS[status]}')
print(f'Took {took_seconds} seconds')

optimal_makespan = int(solver.ObjectiveValue()) + 1
if status == cp_model.OPTIMAL:
    print(f'Optimal makespan: {optimal_makespan}')

import json


today = datetime.date.today()
day = datetime.timedelta(days=1)
gantt_list = []
gantt_links = []
gantt_data = {
    'data': gantt_list,
    'links': gantt_links,
}
for j, job in enumerate(jobs):
    ts = [t for s, area in enumerate(areas) for t in range(optimal_makespan) if j in jobs_per_area[s] and solver.Value(T[j, s]) == t]
    duration = max(ts) - min(ts) + 1
    gantt_list.append({
        "job_id": j,
        "text": job['name'],
        "start_date": format_date(today + day * min(ts)),
        "start_date_obj": today + day * min(ts),
        "duration": duration,
        "progress": 1,
        "open": True,
    })

gantt_list.sort(key=lambda x: x['start_date_obj'])
for i, x in enumerate(gantt_list):
    gantt_list[i]['id'] = i+1
    del gantt_list[i]['start_date_obj']

# Data for google chart
google_chart_rows = []
for s, area in enumerate(areas):
    for j in jobs_per_area[s]:
        job = jobs[j]
        for t in range(optimal_makespan):
            if solver.Value(T[j, s]) == t:
                google_chart_rows.append(
                    [area['name'], job['name'], today + day * t, today + day * (t+1)],
                )

with open('mytestdata.js', 'w+') as f:
    f.write(f'var projects_with_milestones = {json.dumps(gantt_data, ensure_ascii=False)};\n')
    google_chart_rows_str = json.dumps(google_chart_rows, ensure_ascii=False, cls=DateJSONEncoder).replace('"@@', '').replace('@@"', '')
    f.write(f'var google_chart_rows = {google_chart_rows_str};\n')

solver status: infeasible
Took 0.565328 seconds


IndexError: list index out of range

In [50]:
s

15

In [ ]:
ended = datetime.datetime.now()

In [ ]:
ended - started

In [ ]:
for s in range(area_count):
    print(f'Area {s}: ', end='')
    for t in range(optimal_makespan):
        for j in jobs_per_area[s]:
            job = jobs[j]
            if solver.Value(T[j, s]) == t:
#                 print(f'- {job["name"]:3} ', end='')
                print(f'{j:2} ', end='')
                break
        else:
            print(f'  ', end='')
    print('')

In [ ]:
import xlwt

book = xlwt.Workbook()
sheet1 = book.add_sheet('Sheet 1')


for t in range(optimal_makespan):
    sheet1.write(0, t + 1, f'Day {t}')
for s, area in enumerate(areas):
    sheet1.write(s + 1, 0, area['name'])
    for t in range(optimal_makespan):
        for j in jobs_per_area[s]:
            job = jobs[j]
            if solver.Value(T[j, s]) == t:
                st = xlwt.easyxf('pattern: pattern solid;')
                st.pattern.pattern_fore_colour = j + 2  # because 0 is black and 1 is white
                sheet1.write(s + 1, t + 1, job["name"], st)
                break
book.save('result.xls')